In [1]:
spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1749239336629_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'3.5.5-amzn-0'

In [5]:
df = spark.read.csv(
    "s3://nyc-taxi-final-megha/cleaned/cleaned_taxi_data.csv",
    header=True,
    inferSchema=True
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.show()
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|pickup_hour|       tip_percent|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------+------------------+
|       1| 2025-03-01 00:17:16|  2025

In [3]:
df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, VendorID: string, passenger_count: string, trip_distance: string, RatecodeID: string, store_and_fwd_flag: string, PULocationID: string, DOLocationID: string, payment_type: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, congestion_surcharge: string, Airport_fee: string, cbd_congestion_fee: string, pickup_hour: string, tip_percent: string]

In [5]:
df.isnull().sum()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'isnull'
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1749514643032_0001/container_1749514643032_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 3130, in __getattr__
    raise AttributeError(
AttributeError: 'DataFrame' object has no attribute 'isnull'



In [6]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'count' is not defined
Traceback (most recent call last):
  File "<stdin>", line 1, in <listcomp>
NameError: name 'count' is not defined



In [6]:
from pyspark.sql.functions import unix_timestamp, col

# Adding trip duration and filter unreasonable values
df_clean = df.withColumn(
    "trip_duration",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)

# Filter: keeping rows with positive duration and fare
df_clean = df_clean.filter(
    (col("trip_duration") > 0) & (col("fare_amount") > 0)
)

# Previewing cleaned data
df_clean.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_duration", "fare_amount").show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|     trip_duration|fare_amount|
+--------------------+---------------------+------------------+-----------+
| 2025-03-01 00:17:16|  2025-03-01 00:25:52|               8.6|        7.9|
| 2025-03-01 00:37:38|  2025-03-01 00:43:51| 6.216666666666667|        6.5|
| 2025-03-01 00:24:35|  2025-03-01 00:39:49|15.233333333333333|       14.9|
| 2025-03-01 00:56:16|  2025-03-01 01:01:35| 5.316666666666666|        7.2|
| 2025-03-01 00:01:44|  2025-03-01 00:10:00| 8.266666666666667|        8.6|
+--------------------+---------------------+------------------+-----------+
only showing top 5 rows

In [7]:
df_clean.write.mode("overwrite").parquet("s3://nyc-taxi-final-megha/transformed/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.sql.functions import hour, col

df_ml = df_clean.withColumn("pickup_hour", hour("tpep_pickup_datetime"))
df_ml.select("fare_amount", "trip_distance", "trip_duration", "passenger_count", "pickup_hour").show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+------------------+---------------+-----------+
|fare_amount|trip_distance|     trip_duration|passenger_count|pickup_hour|
+-----------+-------------+------------------+---------------+-----------+
|        7.9|          0.9|               8.6|            1.0|          0|
|        6.5|          0.6| 6.216666666666667|            1.0|          0|
|       14.9|         1.94|15.233333333333333|            1.0|          0|
|        7.2|         0.95| 5.316666666666666|            2.0|          0|
|        8.6|          1.5| 8.266666666666667|            1.0|          0|
+-----------+-------------+------------------+---------------+-----------+
only showing top 5 rows

In [12]:
%pip install numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.ml.feature import VectorAssembler




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1749524253299_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
feature_cols = ["trip_distance", "trip_duration", "passenger_count", "pickup_hour"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_ready = assembler.transform(df_ml).select("features", "fare_amount")
df_ready.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|            features|fare_amount|
+--------------------+-----------+
|   [0.9,8.6,1.0,0.0]|        7.9|
|[0.6,6.2166666666...|        6.5|
|[1.94,15.23333333...|       14.9|
|[0.95,5.316666666...|        7.2|
|[1.5,8.2666666666...|        8.6|
+--------------------+-----------+
only showing top 5 rows

In [10]:
# Spliting the data into training and testing sets
train_data, test_data = df_ready.randomSplit([0.8, 0.2], seed=42)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="fare_amount")
lr_model = lr.fit(train_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
test_results = lr_model.evaluate(test_data)

print("Root Mean Squared Error (RMSE):", test_results.rootMeanSquaredError)
print("R-squared (R²):", test_results.r2)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE): 16.441178667398017
R-squared (R?): 0.12218806930338488